In [1]:
import pandas as pd
import os
import re
import numpy as np
import lxml
# import seaborn as sns
import matplotlib.pyplot as plt

folder_path = '/Users/bin/Desktop/nls-catalogue-published-material_dc'

file_prefix = 'BIBLIOGRAPHIC_11573881650004341_'

file_end = '.xml'

data = pd.DataFrame()

for i in range(1,52):
    curr_file = file_prefix + str(i) + file_end
    file_path = os.path.join(folder_path, curr_file)

    df = pd.read_xml(file_path)

    data = pd.concat([data, df], ignore_index = True)

data

,title,creator,type,publisher,date,language,subject,description,coverage,relation,rights,identifier,format
0,Travel /,"Leeson, Edward,1947-2011.",text,London J. Murray,1980,eng,"Short stories, English.",None,None,None,None,None,None
1,Resource book of test items in chemistry,"Jenkins, E. W.(Edgar William)",text,London Murray,1981,eng,Chemistry,Bibliography: p15,None,None,None,None,None
2,Arbitration for contractors,"Stephenson, Douglas A.",text,Northwood Books,1982,,None,None,None,None,None,None,None
3,Armorial bearings of the sovereigns of England...,Standing Conference for Local History.,text,London Bedford Square Press [for the] Standing...,1977,eng,Heraldry,Bibliography: p.29-31,None,None,None,None,None
4,"Sharing caring : caring, equal opportunities a...","Thompson, Catherine.",text,Community Care Project,[1985],,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5091422,"Symfoni Nr. 5, Opus 50 = Symphony no. 5, opus ...","Fjeldsøe, Michael.edt",notated music,"Copenhagen : Edition Wilhelm Hansen,",1998.,zxx,Symphonies,"""Critical commentary,"" ""Filiation and evaluati...",None,None,None,None,None
5091423,Music for violin : from 'El Amor brujo' = Musi...,"Falla, Manuel de,1876-1946.Amor brujo.Danza ri...",notated music,"London : Chester Music,",c1996.,zxx,Ballets,Dance of terror (Danza del terror) -- Pantomim...,None,None,None,None,None
5091424,Fox trot ; and Celebration rag : for SATB reco...,"Hilling, Lyndon.",notated music,"London : Theo Wyatt,",c1984.,eng,None,Publ.no.,None,None,None,None,None
5091425,Fieber : Tondichtung : für Tenor und Orcheste...,"Lehár, Franz1870-1948.",notated music,London Glocken Verlag,[1995],ger,None,2827,None,None,None,None,None


# Time Preprocessing

In [2]:
data_year_undefined = data[data.date.isna() == True]

data = data[data.date.isna() == False]

def extract_four_digit_year(value):
    # Check for the presence of 'c', 'ca', or 'circa' at the beginning
    prefix_match = re.match(r'^(c|ca|circa)', str(value), flags=re.IGNORECASE)
    
    # Extract all digits from the value
    digits = re.findall(r'\d', str(value))
    
    # If no digits found or less than 4 digits, return None
    if not digits or len(digits) < 4:
        return None, "unclear uncertainty"
    
    # Join the digits and convert to integer
    year = int(''.join(digits))
    
    # If exactly 4 digits 
    if len(str(year)) == 4 and int(str(year)) > 1000 and int(str(year)) <= 2023:
        if prefix_match:
            return year, "clear uncertainty"
        if '?' in str(value):
            return year, "clear uncertainty"
        if not prefix_match:
            return year, "certainty"
    
    # If more than 4 digits, return the original value with False
    elif len(str(year)) > 4:
        return value, "unclear uncertainty"
    # Otherwise, return None with False
    else:
        return None, "unclear uncertainty"

data['year'], data['certainty'] = zip(*data['date'].apply(extract_four_digit_year))

data = data[data.date.isna() == False]

# retain collections that we are certain about their publication year
data_certain = data[(data.certainty == 'certainty') | (data.certainty == 'clear uncertainty')]


/var/folders/9s/j57t9vn14b16k36gkr9j6y840000gn/T/ipykernel_7150/4138174611.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['year'], data['certainty'] = zip(*data['date'].apply(extract_four_digit_year))
/var/folders/9s/j57t9vn14b16k36gkr9j6y840000gn/T/ipykernel_7150/4138174611.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['year'], data['certainty'] = zip(*data['date'].apply(extract_four_digit_year))


In [3]:
# types = data_certain['type'].tolist()

# len(types)

In [4]:
# unique_types = pd.unique(data_certain['type']).tolist()

# len(unique_types)

In [5]:
# fdist = FreqDist(types)
# fdist = dict(fdist.most_common(101))

# sum(fdist.values())

In [6]:
def type_preprocess(key):
    # Keywords to be removed if found at the end of an element
    # keywords = [".lcgft",".rgenbr", ".gsafd", ".lcsh", ".rvmgf", ".aat",
    #             ".rbprov", ".lcsh.",".gsafd.", ".rbgenr","rbenr", "gsafd", "lcsh", "rvmgf", "rbgenr",
    #            ".fast(ocolc)fst01411641", ".fast(ocolc)fst01411628", ".fast(ocolc)fst01423787"]

    if key is not None:
        key = key.lower()

        if 'périodiques' in key:
            key = 'periodical'
        if 'ressource internet' in key:
            key = 'Internet resource'
    
        # for keyword in keywords:
        #     if key.endswith(keyword):
        #         key = key[:-len(keyword)]
        #         break
    return key

# fdist = {type_preprocess(key): value for key, value in fdist.items()}

# fdist.keys()

In [7]:
categories_1500 = {
    'text': {
        'children':{
        'incunabula':{}
    }
    },
    'visual media': {
            'children':{'cartographic':{}, 'wooden board':{}}
                 
    },
    
    'miscellaneous':{
        'children':{ 'indulgence':{},  'broadside': {}, 'almanac': {}, 'calandar':{}, 'binding':{}      
        }
    },

}

categories_1550 = {
    'text': {
        'children':{
        'incunabula':{}, 'legal works':{}, 'book':{},'annotation':{}, 'marginalia':{},'inscription':{}     
    }
    },
    'visual media': {
            'children':{'cartographic':{},'device':{}, 'still image':{}, 'wooden board':{}}
                 
    },
    'audio': {
        'children': { 'notated music': {},  'sermon':{}
        }
    },
    'miscellaneous':{
        'children':{ 'indulgence':{}, 'calendar':{}, 'broadside': {}, 'proclamation': {}, 'almanac': {},'chapbook':{}, 'binding':{}
                    
        }
    }
}

categories_1600 = {
    'text': {
        'children':{
        'incunabula':{}, 'legal works':{}, 'book':{}, 'marginalia':{}, 
        'poem':{}, 'inscription':{}, 'apologetics':{}, 'grammar':{}, 'catechism':{}
    }
    },
    'visual media': {
            'children':{'cartographic':{},'device':{}, 'still image':{}, 'wooden board':{}, 'painting':{}}
                 
    },
    'audio': {
        'children': { 'notated music': {}, 'verse':{}, 'ballad':{}, 'sermon':{}
        }
    },
    'miscellaneous':{
        'children':{ 'indulgence':{}, 'calendar':{}, 'broadside': {}, 'proclamation': {}, 'almanac': {}, 
                     'chapbook':{}, 'early works':{}, 'printed waste':{}, 'binding':{}
                    
        }
    },
    'undefined': {}
}

categories_1650 = {
    'text': {
        'children':{
        'incunabula':{}, 'legal works':{}, 'newspaper':{}, 'book':{}, 'dissertation':{},'poem':{}, 'satire':{}, 'annotation':{},
        'religious works':{}, 'inscription':{}, 'periodical':{}, 'legislative proceeding':{}
            
    }
    },
    'visual media': {
            'children':{'cartographic':{},'device':{}, 'still image':{}, 'wooden board':{}, 'painting':{}, 'decorated paper':{}}
                 
    },
    'audio': {
        'children': { 'notated music': {}, 'verse':{}, 'ballad':{}, 'presentation':{}, 'sermon':{}
        }
    },
        'IT':{
            'children':{ 'software':{}}
        },
    'miscellaneous':{
        'children':{ 'indulgence':{}, 'broadside': {}, 'proclamation': {}, 'almanac': {}, 'address':{}, 'chapbook':{},
                     'binding':{}, 'label':{}, 'bookplate':{}, 'early works':{}, 'marking':{}, 'play':{}                
        }
    },
    'undefined': {}
}

categories_1700 = {
    'text': {
        'children':{
        'incunabula':{}, 'legal works':{}, 'newspaper':{}, 'book':{}, 'dissertation':{},'poem':{}, 'satire':{}, 'annotation':{},
        'periodical':{}, 'regulation':{},'grammar':{},'session laws':{}
            
    }
    },
    'visual media': {
            'children':{'cartographic':{},'still image':{}, 'wooden board':{}, 'painting':{}, 'decorated paper':{}}
                 
    },
    'audio': {
        'children': { 'notated music': {}, 'verse':{},'ballad':{}, 'presentation':{}, 'narratives':{}, 'sermon':{}, 'elegies':{}
        }
    },
    'miscellaneous':{
        'children':{ 'indulgence':{}, 'broadside': {}, 'proclamation': {}, 'almanac': {}, 'chapbook':{}, 'address':{},
                     'binding':{}, 'label':{}, 'catalog':{},'bookplate':{}, 'early works':{}, 'play':{}, 'comedies':{}               
        }
    },
    'undefined': {}
}

categories_1750 = {
    'text': {
        'children':{
        'incunabula':{}, 'legal works':{}, 'newspaper':{}, 'book':{}, 'dissertation':{},'poem':{}, 'annotation':{},
         'periodical':{}, 'letter':{}, 'prospectus':{}, 'libretto':{}
    }
    },
    'visual media': {
            'children':{'cartographic':{}, 'still image':{}, 'wooden board':{}, 'painting':{}, 'three dimensional object':{}
                       
                       }           
    },
    'audio': {
        'children': { 'notated music': {}, 'verse':{},'ballad':{}, 'narratives':{}, 'speech':{}, 'sermon':{}
        }
    },
    'miscellaneous':{
        'children':{ 'indulgence':{}, 'broadside': {}, 'proclamation': {}, 'almanac': {}, 'chapbook':{},
                     'address':{},'binding':{}, 'label':{}, 'early works':{}, 'advertisement':{},

                    'account':{}, 'bookplate':{}, 'comedies':{},'play':{}, 'catalog':{}               
        }
    },
    'undefined': {}
}

categories_1800 = {
    'text': {
        'children':{
         'legal works':{}, 'newspaper':{},'book':{}, 'dissertation':{},'poem':{},'annotation':{},
         'periodical':{}, 'letter':{}, 'prospectus':{}
    }
    },
    'visual media': {
            'children':{'cartographic':{}, 'still image':{}, 'wooden board':{}, 'three dimensional object':{}                       
                       }
                 
    },
    'audio': {
        'children': { 'notated music': {}, 'verse':{},'ballad':{}, 'speech':{}, 'sermon':{}
        }
    },
    'miscellaneous':{
        'children':{ 'indulgence':{}, 'broadside': {}, 'proclamation': {}, 'almanac': {}, 'chapbook':{},
                     'address':{},'binding':{}, 'label':{}, 'advertisement':{},

                    'account':{}, 'bookplate':{}        
        }
    },
    'undefined': {}
}

categories_1850 = {
    'text': {
        'children':{
         'legal works':{}, 'book':{}, 'dissertation':{},'poem':{},'annotation':{},
         'periodical':{}, 'letter':{}
    }
    },
    'visual media': {
            'children':{'cartographic':{}, 'still image':{}, 'wooden board':{}, 'poster':{},   

                        'playbill':{}
                       }
                 
    },
    'audio': {
        'children': { 'notated music': {}, 'verse':{},'ballad':{}, 'sermon':{}
        }
    },
    'miscellaneous':{
        'children':{ 'indulgence':{}, 'broadside': {}, 'proclamation': {}, 'almanac': {}, 'chapbook':{},
                     'address':{},  'advertisement':{},

                    'account':{},       
        }
    },
    'undefined': {}
}

categories_1900 = {
    'text': {
        'children':{
         'legal works':{}, 'book':{}, 'dissertation':{},'poem':{},'annotation':{},
         'periodical':{}, 'journal':{}, 'fiction':{}, 'history':{},'letter':{}
    }
    },
    'visual media': {
            'children':{'cartographic':{}, 'still image':{}, 'wooden board':{}, 'poster':{},   
                        'playbill':{}, 'three dimensional object':{}
                       }
                 
    },
    'audio': {
        'children': { 'notated music': {}, 'verse':{},'ballad':{}, 'sermon':{}
        }
    },
    'miscellaneous':{
        'children':{ 'indulgence':{}, 'broadside': {}, 'proclamation': {}, 'almanac': {}, 'chapbook':{},
                     'address':{}, 'account':{}, 'theater program':{},
                    
                    'advertisement':{}                          
        }
    },
    'undefined': {}
}

categories_1950 = {
    'text': {
        'children':{
         'book':{}, 'dissertation':{},'poem':{},'annotation':{},
         'periodical':{}, 'journal':{}, 'fiction':{}, 'history':{},'letter':{}
    }
    },
    'visual media': {
            'children':{'cartographic':{}, 'still image':{}, 'wooden board':{}, 'poster':{}, 'three dimensional object':{},

                        'playbill':{}
                       }
                 
    },
    'audio': {
        'children': { 'notated music': {}, 'recording':{},'verse':{},'ballad':{}, 'sermon':{}
        }
    },
    'miscellaneous':{
        'children':{ 'indulgence':{}, 'broadside': {}, 'proclamation': {}, 'almanac': {}, 'chapbook':{},
                     'address':{}, 'account':{}, 'theater program':{},'advertisement':{}                          
        }
    },
    'undefined': {}
}

categories_2000 = {
    'text': {
        'children':{
         'book':{}, 'newspaper':{},'poem':{},'annotation':{}, 'stories':{}, 'reader':{},
         'periodical':{}, 'journal':{}, 'fiction':{}, 'history':{}, 'bildungsromane':{}, 'tale':{}
    }
    },
    'visual media': {
            'children':{'cartographic':{}, 'moving image':{},'still image':{}, 'poster':{}, 'three dimensional object':{},
                       }
                 
    },
    'audio': {
        'children': { 'notated music': {}, 'recording':{},'ballad':{}
        }
    },
    'IT': {
        'children': { 'software':{}, 'Internet resource':{}
        }
    },
    'miscellaneous':{
        'children':{ 'broadside': {}, 'proclamation': {}, 'almanac': {}, 'chapbook':{},
                      'theater program':{}                           
        }
    },
    'undefined': {}
}

categories_2022 = {
    'text': {
        'children':{
        'book': {}, 'stories': {}, 'fiction': {}, 'tale': {}, 'novel':{}, 'annotation':{},
             'history': {}, 'bildungsromane': {}, 'reader': {}, 'journal': {}, 'periodical': {} #, 'périodique': {}
    }
    },
    'visual media': {
            'children':{'three dimensional object': {},'cartographic': {},'moving image': {}, 'still image': {}, 'poster': {}}              
    },
    'audio': {
        'children': {
            'recording': {}, 'notated music': {}, 'ballad': {}
        }
    },
    'IT': {
        'children': {
            'software':{}, 'Internet resource':{}
        }
    },
    'miscellaneous':{
        'children':{
            'broadside': {}, 'proclamation': {}, 'almanac': {}, 'theater program': {}, 'chapbook':{}
        }
    },
    'undefined': {}
}


In [8]:
categories = [categories_1500, categories_1550, categories_1600, categories_1650, categories_1700, categories_1750,
              categories_1800, categories_1850, categories_1900, categories_1950, categories_2000, categories_2022]

In [9]:
from collections import deque
from nltk import FreqDist


def find_deepest_category(entry, categories):
    if entry is None:
        return ['Undefined']

    entry = entry.lower()  # Convert the entry to lowercase for case-insensitive matching
    queue = deque([({'children': categories}, [])])  # Initialize the queue
    deepest_match = ['Undefined']
    longest_match_length = 0  # Track the length of the longest match

    while queue:
        current_item, path = queue.popleft()
        current_level = current_item.get('children', {})

        for category, details in current_level.items():
            category_lower = category.lower()  # Convert category name to lowercase for case-insensitive matching            
            # Check if the category name is a substring of the entry
            if category_lower in entry:
                match_length = len(category_lower)
                # Update the deepest match if this match is longer than any previous match
                if match_length > longest_match_length:
                    deepest_match = path + [category]
                    longest_match_length = match_length

            # If there are children, add them to the queue to explore further
            if 'children' in details:
                queue.append((details, path + [category]))

    return deepest_match

def create_dataframe(dataframe,categories):
    types = dataframe['type'].tolist()
    
    if len(types) < 101:
        fdist = FreqDist(types)
        fdist = dict(fdist.most_common(len(types)))
    else:
        fdist = FreqDist(types)
        fdist = dict(fdist.most_common(101))

    fdist = {type_preprocess(key): value for key, value in fdist.items()}
    # print(fdist)
    
    hierarchy_info = []
    
    for k, v in fdist.items():
        if k is None:
            row = {'Level 0': 'Undefined', 'Level 1': ' ',  'Key': ' ', 'Value': v}
            hierarchy_info.append(row)
        else:
            if k != 'text':
                match = find_deepest_category(k, categories)
    
                if len(match) == 1:
                    row = {'Level 0': match[0], 'Level 1': '', 'Key': k, 'Value': v}
                    hierarchy_info.append(row)
                
                elif len(match) == 2:
                    row = {'Level 0': match[0], 'Level 1': match[1], 'Key': k, 'Value': v}
                    hierarchy_info.append(row)

    hierarchies = pd.DataFrame(hierarchy_info)

    return hierarchies


In [15]:
import dash
from dash import Dash, html, dcc, Input, Output
import plotly.express as px


time = [1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,2022]

app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='sunburst-chart'),
    dcc.Slider(
        id='year-slider',
        min=min(time),
        max=max(time),
        value=min(time),
        marks={str(year): str(year) for year in time},
        step=None  # Prevent intermediate values
    ),
    html.Button("Save Chart", id="btn-save")
])

@app.callback(
    Output('sunburst-chart', 'figure'),
    [Input('year-slider', 'value')]
)
def update_chart(selected_year):
    index = time.index(selected_year)
    hierarchy = create_dataframe(data_certain[data_certain.year <= selected_year], categories[index])
    fig = px.sunburst(
        hierarchy, 
        path=['Level 0', 'Level 1'],
        values='Value',
        labels='Key',
        title=f"Sunburst Chart of Categories for {selected_year}",
        width=1000
    )
    fig.update_layout(
        width=800,
        height=600,
        margin=dict(t=50, l=25, r=25, b=25)
    )
    # Store the figure in the global context (this is not the best practice but works for simple use cases)
    global saved_fig
    saved_fig = fig
    return fig

@app.callback(
    Output('btn-save', 'children'),  # Update the button text after saving
    [Input('btn-save', 'n_clicks')],
    prevent_initial_call=True
)
def save_figure_to_html(n_clicks):
    if n_clicks is not None:
        saved_fig.write_html('saved_sunburst_chart.html')
        return "Chart Saved!"
    return "Save Chart"

if __name__ == '__main__':
    app.run_server(debug=True)

In [13]:
# import ipywidgets as widgets
# from IPython.display import display, clear_output
# from IPython.display import HTML
# import plotly
# import plotly.express as px
# import json

# time = [1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, 1950, 2000, 2022]

# def update_graph(change):
#     selected_year = change.new
#     index = time.index(selected_year)
#     hierarchy = create_dataframe(data_certain[data_certain.year <= selected_year], categories[index])
#     fig = px.sunburst(
#         hierarchy,
#         path=['Level 0', 'Level 1'],
#         values='Value',
#         labels='Key',
#         title=f"Sunburst Chart of Categories for {selected_year}"
#     )
#     fig.update_layout(
#         width = 800, height = 600,
#         legend=dict(
#         title="Legend",  # Optional: add a title to the legend
#         orientation="h",  # Horizontal legend
#         yanchor="bottom",
#         y=0.01,  # Adjust the y position, where 0 is the bottom of the plot area
#         xanchor="right",
#         x=1  # Adjust the x position, where 1 is the right of the plot area
#     ),
#     margin=dict(t=50, l=25, r=25, b=25))  # Adjust margins to ensure legend is visible)
#     with out:
#         clear_output(wait=True)
#         display(fig)

# slider = widgets.IntSlider(
#     value=min(time),
#     min=min(time),
#     max=max(time),
#     step=50,  # Assuming you want steps to be in increments of 50 years
#     description='Year:',
#     continuous_update=False
# )

# out = widgets.Output()
# display(slider, out)
# slider.observe(update_graph, names='value')

IntSlider(value=1500, continuous_update=False, description='Year:', max=2022, min=1500, step=50)

Output()

In [28]:
# # HTML Template
# html_template = f"""
# <!DOCTYPE html>
# <html>
# <head>
#     <meta charset="UTF-8">
#     <title>Interactive Sunburst Chart</title>
#     <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
# </head>
# <body>
#     <input type="range" id="year-slider" min="1500" max="2022" value="1500" step="50">
#     <div id="chart"></div>
#     <script>
#         var initialConfig = {fig_json}; // Embed the figure JSON directly

#         function updateFigure(selectedYear) {{
#             // Deep copy to prevent mutation
#             var updatedConfig = JSON.parse(JSON.stringify(initialConfig));
#             updatedConfig.layout.title.text = "Sunburst Chart for Year " + selectedYear; // Dynamically update the title
#             Plotly.react('chart', updatedConfig.data, updatedConfig.layout);
#         }}

#         // Initial rendering of the plot
#         Plotly.newPlot('chart', initialConfig.data, initialConfig.layout);

#         // Add event listener to the slider
#         document.getElementById('year-slider').addEventListener('input', function() {{
#             updateFigure(this.value);
#         }});
#     </script>
# </body>
# </html>
# """

# # Display the HTML in the notebook for preview
# from IPython.core.display import display, HTML
# display(HTML(html_template))

/tmp/ipykernel_224093/1419572229.py:36: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display



In [43]:
# Write the HTML string to an HTML file
# with open('interactive_sunburst_chart.html', 'w') as f:
#     f.write(html_template)

# print("HTML file created successfully!")

HTML file created successfully!


In [20]:
# hierarchies = create_dataframe(data_certain[data_certain.year <= 1550],categories_1550)

# import plotly.express as px

# # Creating the sunburst chart
# fig = px.sunburst(
#     hierarchies, 
#     path=['Level 0', 'Level 1'],  # The hierarchy of your categories
#     values='Value',  # The values for each leaf node
#     labels='Key',  # The names for each leaf node
#     title="Sunburst Chart of Categories",  # Optional: Adds a title to your chart
#     width = 1000
# )

# # Set the size of the figure here
# fig.update_layout(
#     width=800,   # Adjust the width as needed
#     height=600,  # Adjust the height as needed
#     margin=dict(t=50, l=25, r=25, b=25)  # Adjust the margins if needed
# )

# # Show the figure
# fig.show()

# Publication Type Preprocessing

In [ ]:
# import re

# def split_words_in_list(words_list):
#     result = []
#     for text in words_list:
#         # Split the text based on spaces or transitions from lowercase to uppercase
#         words = re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$)', text)
#         result.append(words)
#     return result

# unique_types = unique_types[np.logical_not(np.equal(unique_types, None))]
# cleaned_unique_types = split_words_in_list(unique_types)

In [ ]:
# def categorize_data(processed_words_list, book_keywords, academic_keywords, 
#                     audio_keywords, art_keywords, computer_keywords, data_keywords):
#     categorized_data = {}
#     for entry in processed_words_list:
#         is_academic = any(word.lower() in academic_keywords or any(keyword.lower() in word.lower() for keyword in academic_keywords) for word in entry)
#         if is_academic:
#             categorized_data[' '.join(entry)] = 'academic'
#         else:
#             is_recording = any(word.lower() in audio_keywords or any(keyword.lower() in word.lower() for keyword in audio_keywords) for word in entry)
#             if is_recording:
#                 categorized_data[' '.join(entry)] = 'recording'
#             else:
#                 is_art = any(word.lower() in art_keywords or any(keyword.lower() in word.lower() for keyword in art_keywords) for word in entry)
#                 if is_art:
#                     categorized_data[' '.join(entry)] = 'art'
#                 else:
#                     is_computer = any(word.lower() in computer_keywords or any(keyword.lower() in word.lower() for keyword in computer_keywords) for word in entry)
#                     if is_computer:
#                         categorized_data[' '.join(entry)] = 'computer'
#                     else:
#                         is_data = any(word.lower() in book_keywords or any(keyword.lower() in word.lower() for keyword in data_keywords) for word in entry)
#                         if is_data:
#                             categorized_data[' '.join(entry)] = 'data'
#                         else:    
#                             is_book = any(word.lower() in book_keywords or any(keyword.lower() in word.lower() for keyword in book_keywords) for word in entry)
#                             if is_book:
#                                 categorized_data[' '.join(entry)] = 'book'
#                             else:
#                                 categorized_data[' '.join(entry)] = 'unknown'
    
#     return categorized_data

# # Example keywords related to books
# book_keywords = [
#     "book", "story", "stories", "fiction", "biography", "tale", "interpretation",
#     "journal", "broadside", "proclamation", "read", "translation", "almanac", 'Brtoadsides',
#     "periodi", "newspaper", "magazine","novel", "print", "text", "poetry", "poem","poezja", "sonnet","humor", "guide", "eriodical",
#     "memoir", "manual", "encyclopedia", "dictionar","recipe", "binding", "publica", "publish", "newsletter", "writing", "constitution"
# ]

# academic_keywords = [
#     "paper", "proceeding", "dissertation", "academic", "conference", "thesis", "annotation", "presentation", "sample", "proof", "guildeline", "treaties",
#      "exam", "guide", "law", "essay", "manuscript", "studies", "report", "literature", "quotation", "teach", "theory", "anecdote", "archive", "review", "reference", "outline",
#     "literary", "document", "bibliograph","criticism", "lecture", "abstracts", "survey", "work", "exam","problem", "question", "grammar","histor", "scientific"
# ]

# audio_keywords = ["song","sing","music","recording", "recorded","score","sound","audio", "symphon", "sonata","anthem","opera","rhyme","ballad","folklore", "speech",
#                   "narrat","commenta", "radio", "jazz", "concert","film","video", "dialog", "interview"]

# art_keywords = ["art","sculpture","paint","draw","theatre","theater","play","drama", "advertise", "portrait",
#                 "picture", "pictorial", "image","photo", "cartoon", "comic","manga", "poster", "lyric", "color", "game"]


# computer_keywords = ["software","computer", "http", "internet", "disc", "disk","code",
#                      "web", "program", "database","electronic", "online", "Elektronische"]

# data_keywords = ["map", "statisti", "chart", "graph", "atlas", "sheet", "address",
#                  "directory", "directories", "list","catalog","collection","form" ]

# # Apply categorization function to the processed words list
# categorized_data = categorize_data(cleaned_unique_types, book_keywords, academic_keywords,
#                                    audio_keywords, art_keywords, computer_keywords, data_keywords)


# # Get the number of items with the value of "unknown"
# unknown_count = sum(1 for category in categorized_data.values() if category == 'unknown')

# print("Number of items with value 'unknown':", unknown_count)

In [ ]:
# unknown_keys = [key for key, value in categorized_data.items() if value == 'unknown']

# unknown_keys